In [1]:
!pip install datasets
!pip install transformers[sentencepiece]
!pip install --upgrade transformers
!pip install einops
!pip install openai
!pip install evaluate
!pip install torch
!pip install huggingface_hub
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 75.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 81.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 35.6 MB/s eta 0:00:00
   ━━━━

In [15]:
from datasets import load_dataset, load_from_disk, concatenate_datasets
from functools import partial
import pandas as pd
import numpy as np

from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login

import evaluate
import torch
import torch.nn as nn

import zipfile
import tempfile
import shutil
import os
import re

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
def load_and_extract_dataset(zip_path):
    # Create a temporary directory
    temp_dir = '/tmp/dataset_extracted'
    
    # Ensure the directory is clean
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
    os.makedirs(temp_dir)
    
    # Unzip the file to the temporary directory
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)

    # Load the dataset from the extracted directory
    dataset = load_from_disk(temp_dir)

    return dataset

In [9]:
dataset = load_and_extract_dataset("dataset (1).zip")
# tokenized_datasets = load_and_extract_dataset("tokenized_datasets2.zip")

In [10]:
def sampleDataset(samples):
    train_dataset = dataset['train']

    dataset_sentiment_0 = train_dataset.filter(lambda x: x['label'] == 0)
    dataset_sentiment_1 = train_dataset.filter(lambda x: x['label'] == 1)

    dataset_sampled_0 = dataset_sentiment_0.shuffle().select(range(samples//2))
    dataset_sampled_1 = dataset_sentiment_1.shuffle().select(range(samples//2))

    dataset_combined = concatenate_datasets([dataset_sampled_0, dataset_sampled_1])

    sampled_dataset = dataset_combined.shuffle()

    print("Positive: ", sum(1 for example in sampled_dataset if example['label'] == 1))
    print("Negative: ", sum(1 for example in sampled_dataset if example['label'] == 0))
    print(sampled_dataset)

    update_fn = partial(update_column, positivePrompt = positivePrompt, negativePrompt = negativePrompt)
    sampled_dataset = sampled_dataset.map(update_fn, batched=False)
    
    return sampled_dataset


def update_column(example, positivePrompt, negativePrompt):
    usedPrompt = positivePrompt if example["label"] == 1 else negativePrompt
    # usedString = '\n\nPositive: ' if example ["label"] == 1 else '\n\nNegative: "' 
    
    example["text"] = usedPrompt  + example["text"] + '"'
    return example


In [11]:
positivePrompt = '''
Generate a positive social media tweet on a specific topic. Ensure your tweet is enclosed in straight double quotation marks. Provide ONLY one tweet. The positive tweet should express enthusiasm or praise. 

Positive: "'''

In [12]:
negativePrompt = '''
Generate a negative social media tweet on a specific topic. Ensure your tweet is enclosed in straight double quotation marks and separated by a colon. Provide ONLY one tweet. The negative tweet should express convey criticism or disappointment. 

Negative: "'''

In [13]:
sampled_dataset = sampleDataset(1000)

Filter:   0%|          | 0/1600000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1600000 [00:00<?, ? examples/s]

Positive:  500
Negative:  500
Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [16]:
sampled_dataset["text"][8]

'\nGenerate a positive social media tweet on a specific topic. Ensure your tweet is enclosed in straight double quotation marks. Provide ONLY one tweet. The positive tweet should express enthusiasm or praise. \n\nPositive: "@LADRACU hey I\'m in the process of doing an ap econ final right now. I still have to write frqs "'

In [16]:
# 100 max tokens on negative => Perfect generation

In [32]:
def deleteIrrelevent(s, prompt, isPositive):
    s = s.replace(prompt, '')

    # Find the index of the second double quotation mark
    first_quote_index = s.find('"')
    if first_quote_index == -1:
        # No double quotation mark found
        return s
    
    second_quote_index = s.find('"', first_quote_index + 1)
    if second_quote_index == -1:
        # Only one double quotation mark found
        return s

    findTweets(s, prompt, 
    
    # Return the substring up to the second double quotation mark
    return s[:second_quote_index + 1]

SyntaxError: '(' was never closed (3026340403.py, line 15)

In [26]:
output = promptModel(negativePrompt)
x = output[0]

print(x)


Generate a negative social media tweet on a specific topic. Ensure your tweet is enclosed in straight double quotation marks and separated by a colon. Provide ONLY one tweet. The negative tweet should express convey criticism or disappointment. 

Negative: "Absolutely appalling to see the state of #ClimateAction being ignored at this crucial time for our planet!" ::: Climate Change Negativity ::


In [27]:
y = findTweets(x, False)
print(y)

Absolutely appalling to see the state of #ClimateAction being ignored at this crucial time for our planet!


In [17]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "microsoft/Phi-3-mini-4k-instruct"
# model_name = "mistralai/Mistral-7B-Instruct-v0.3"
# model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# model_name = "distilbert/distilbert-base-uncased"
# model_name = "mistralai/Mistral-Nemo-Instruct-2407"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

torch.cuda.empty_cache()

if(torch.cuda.is_available()):
  model = model.cuda()

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [79]:
def count_rows_with_hashtags(dataset, column_name):
    """
    Count how many rows in the Dataset contain a hashtag in the specified column.
    
    Parameters:
    dataset (Dataset): The Dataset to search through.
    column_name (str): The name of the column to search for hashtags.
    
    Returns:
    int: The count of rows containing hashtags in the specified column.
    """
    # Check if the column exists in the dataset
    if column_name not in dataset.column_names:
        raise ValueError(f"Column '{column_name}' does not exist in the dataset.")
    
    # Define a function to check if a string contains a hashtag
    def contains_hashtag(text):
        return isinstance(text, str) and '#' in text
    
    # Apply the function to the specified column and sum the results
    count = sum(contains_hashtag(row[column_name]) for row in dataset)
    
    return count

count_rows_with_hashtags(dataset["train"], "text")

36812

In [75]:
generateAndExtractTweets(positivePrompt, 1, 100)



STEP 1: INITAL PROMPT
Generated Text: 
Generate a positive social media tweet on a specific topic. Ensure your tweet is enclosed in straight double quotation marks. Provide ONLY one tweet. The positive tweet should express enthusiasm or praise. 

Positive: "Just witnessed an incredible act of kindness! Our community spirit truly shines when we lift each other up, and I'm inspired by the good everyone does every day!" #CommunityKindness


STEP 2: TWEET SEARCH


STEP 3: RE PROMPTING
Generated Text: Just witnessed an incredible act of kindness! Our community spirit truly shines when we lift each other up, and I'm inspired by the good everyone does every day! 


("Just witnessed an incredible act of kindness! Our community spirit truly shines when we lift each other up, and I'm inspired by the good everyone does every day! ",
 tensor([[[13.9795, 15.5622, 16.6405,  ..., 12.0748, 12.0724, 12.0755],
          [31.0314, 32.5049, 34.0404,  ..., 28.1738, 28.1774, 28.1815],
          [32.8489, 34.1223, 36.8424,  ..., 30.7938, 30.7961, 30.8001],
          ...,
          [33.9708, 35.1541, 36.6095,  ..., 26.5326, 26.5384, 26.5402],
          [31.8688, 33.0583, 34.7031,  ..., 26.8111, 26.8144, 26.8150],
          [22.0982, 24.4024, 22.4930,  ..., 13.8696, 13.8738, 13.8664]]],
        device='cuda:0'))

In [60]:
def promptModel(prompt, length):
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    attention_mask = inputs['attention_mask'].to(device)

    outputs = model.generate(
        inputs['input_ids'],
        do_sample=True,
        attention_mask=attention_mask,
        num_return_sequences=1,
        max_length=length,
        temperature = 0.9, 
        top_p = 0.9, 
        repetition_penalty=1.2,
        output_scores=True,
        return_dict_in_generate=True
    )

    # Decode the generated text
    text = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    print("Generated Text:", text)

    with torch.no_grad():
        # Forward pass to get logits
        logits = model(outputs.sequences).logits


    return text, logits

In [30]:
def findTweets(generated_text, isPositive):
    output = generated_text.replace('“', '"').replace('”', '"')

    if(isPositive):
        positive_match = re.search(r'Positive:\s*"\s*([^"]*)\s*"', output, re.DOTALL)
        positive_tweet = positive_match.group(1).strip() if positive_match else "-1"
        return positive_tweet

    else:
        negative_match = re.search(r'Negative:\s*"\s*([^"]*)\s*"', output, re.DOTALL)
        negative_tweet = negative_match.group(1).strip() if negative_match else "-1"
        return negative_tweet

In [74]:
def generateAndExtractTweets(prompt, label, length):
            
    while True:
        print("\n\nSTEP 1: INITAL PROMPT")
        text, logits = promptModel(prompt, length)
        print("\n\nSTEP 2: TWEET SEARCH")
        tweet = findTweets(text, label == 1)
        
        if tweet != "-1" and len(tweet) > 0:
            print("\n\nSTEP 3: RE PROMPTING")
            length = len(tokenizer.encode(tweet, add_special_tokens=True))
            text, logits = promptModel(tweet, length + 1)  
            return text, logits

        print("\n\n\nretry!!")


In [20]:
import torch.optim as optim
from transformers import AdamW

class Discriminator(nn.Module):
  def __init__(self, vocab_size, embed_size):
    super(Discriminator, self).__init__()
    self.embedding = nn.Embedding(vocab_size, embed_size)
    self.lstm = nn.LSTM(embed_size, 128, batch_first=True)
    self.fc = nn.Linear(128, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, input_ids):
    embeds = self.embedding(input_ids)
    _, (hidden, _) = self.lstm(embeds)
    output = self.fc(hidden[-1])
    return self.sigmoid(output)
#generator and discriminator instantiation
generator = model

if(torch.cuda.is_available()):
  discriminator = Discriminator(len(tokenizer), 768).cuda()
else:
    discriminator = Discriminator(len(tokenizer), 768)

#optimizers
optimizerG = AdamW(generator.parameters(), lr=5e-5)
optimizerD = optim.Adam(discriminator.parameters(), lr=5e-5)
#loss function
criterion = nn.BCEWithLogitsLoss()
data_loader = torch.utils.data.DataLoader(sampled_dataset, batch_size=1, shuffle=True)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [44]:
for batch in data_loader:
    print(batch.keys())
    break

dict_keys(['text', 'label'])


In [ ]:
i=0
for batch in data_loader:
    print(batch) 
    i+=1
    if i > 3:
        break

In [35]:
for batch in data_loader:
        
        # Convert the input batch into a PyTorch tensor and move to GPU
    padded_inputs = tokenizer(batch['text'], padding=True, return_tensors="pt", truncation=True)
    real_text = padded_inputs['input_ids'].cuda()
    print(real_text)
    break

tensor([[128000,    198,  32215,    264,   8389,   3674,   3772,  12072,    389,
            264,   3230,   8712,     13,  30379,    701,  12072,    374,  44910,
            304,   7833,   2033,  55331,  15785,    323,  19180,    555,    264,
          15235,     13,  40665,  27785,    832,  12072,     13,    578,   8389,
          12072,   1288,   3237,  20599,  19347,    477,  41698,     13,   4815,
          39589,     25,   8594,   1445,  28451,  30123,   7008,   2771,   1587,
           3194,    499,      0,    220,    330]], device='cuda:0')


In [ ]:
import torch
import numpy as np
import os

# For more accurate error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ["PYTORCH_USE_CUDA_DSA"] = "1"

batches = 1
epochs = 5
for epoch in range(epochs):
    for batch in data_loader:
        
        # Convert the input batch into a PyTorch tensor and move to GPU
        padded_inputs = tokenizer(batch['text'], padding=True, return_tensors="pt", truncation=True)
        real_text = padded_inputs['input_ids'].cuda()

        # Generate fake text logits using noise input
        noise = torch.randint(0, len(tokenizer), (1, 16)).cuda()
        outputs = generator(noise)

        promptUsed = positivePrompt if batch["label"] == 1 else negativePrompt
        labelUsed = batch["label"]
        
        fake_logits_raw = generateAndExtractTweets(promptUsed, labelUsed).cuda()
        fake_logits = fake_logits_raw.argmax(dim=-1).detach().cuda()
        
        # Reset gradients for discriminator
        discriminator.zero_grad()

        # Create real labels tensor and move to GPU
        real_labels = torch.ones((real_text.size(0), 1), dtype=torch.float).cuda()

        # Get discriminator's prediction on real text and calculate loss
        real_output = discriminator(real_text)
        lossD_real = criterion(real_output.view(-1, 1), real_labels)
        
        # Reshape fake logits to ensure correct shape
        fake_logits = fake_logits.view(-1, 1).cuda()

        # Get discriminator's prediction on fake text and calculate loss
        fake_output = discriminator(fake_logits)
        
        # Create fake labels tensor and move to GPU
        fake_labels = torch.zeros((fake_output.size(0), 1), dtype=torch.float).cuda()
        
        lossD_fake = criterion(fake_output, fake_labels)

        # Combine real and fake losses for the discriminator
        lossD = lossD_real + lossD_fake
        lossD.backward()
        
        torch.nn.utils.clip_grad_norm_(discriminator.parameters(), max_norm=1.0)
        
        optimizerD.step()

        # Reset gradients for generator
        generator.zero_grad()

        # Generate new fake text logits
        with torch.no_grad():
            fake_logits_raw = generateAndExtractTweets(promptUsed, labelUsed).cuda()
        fake_logits = fake_logits_raw.argmax(dim=-1).detach().cuda()

        # Get discriminator's assessment of the newly generated fake data
        fake_output = discriminator(fake_logits.view(-1, 1).cuda())
        real_labels = torch.ones((fake_output.size(0), 1), dtype=torch.float).cuda()

        # Calculate the generator's loss

        lossG = criterion(fake_output.view(-1, 1), real_labels)
        lossG.backward()
        
        torch.nn.utils.clip_grad_norm_(generator.parameters(), max_norm=1.0)

        optimizerG.step()
        
        print(f'Epoch [{epoch}/{epochs}] Loss_D: {lossD.item():.4f} Loss_G: {lossG.item():.4f}')
        print("BATCH NUMBER " + str(batches))
        batches+=1
        del padded_inputs, real_text, noise, outputs, fake_logits, fake_logits_raw, real_labels, real_output, fake_labels, fake_output
        # gc.collect(generation=2)
        torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Epoch [0/5] Loss_D: 1.4515 Loss_G: 0.4699
BATCH NUMBER 1


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Epoch [0/5] Loss_D: 1.4717 Loss_G: 0.4698
BATCH NUMBER 2


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Epoch [0/5] Loss_D: 1.1957 Loss_G: 0.5519
BATCH NUMBER 506


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Epoch [0/5] Loss_D: 1.2026 Loss_G: 0.5538
BATCH NUMBER 507


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


## Working training Loop!

In [ ]:
import torch
import numpy as np
import os

# For more accurate error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ["PYTORCH_USE_CUDA_DSA"] = "1"

batches = 1
epochs = 5
for epoch in range(epochs):
    for batch in data_loader:
        
        # Convert the input batch into a PyTorch tensor and move to GPU
        padded_inputs = tokenizer(batch['text'], padding=True, return_tensors="pt", truncation=True)
        real_text = padded_inputs['input_ids'].cuda()

        # Generate fake text logits using noise input
        noise = torch.randint(0, len(tokenizer), (1, 16)).cuda()
        outputs = generator(noise)
        fake_logits = outputs.logits.argmax(dim=-1).detach().cuda()  # Detach to avoid backpropagation
        fake_logits_raw = outputs.logits.cuda()  # Raw logits before argmax for shape consistency
        
        # Reset gradients for discriminator
        discriminator.zero_grad()

        # Create real labels tensor and move to GPU
        real_labels = torch.ones((real_text.size(0), 1), dtype=torch.float).cuda()

        # Get discriminator's prediction on real text and calculate loss
        real_output = discriminator(real_text)
        lossD_real = criterion(real_output.view(-1, 1), real_labels)
        
        # Reshape fake logits to ensure correct shape
        fake_logits = fake_logits.view(-1, 1).cuda()

        # Get discriminator's prediction on fake text and calculate loss
        fake_output = discriminator(fake_logits)
        
        # Create fake labels tensor and move to GPU
        fake_labels = torch.zeros((fake_output.size(0), 1), dtype=torch.float).cuda()
        
        lossD_fake = criterion(fake_output, fake_labels)

        # Combine real and fake losses for the discriminator
        lossD = lossD_real + lossD_fake
        lossD.backward()
        
        torch.nn.utils.clip_grad_norm_(discriminator.parameters(), max_norm=1.0)
        
        optimizerD.step()

        # Reset gradients for generator
        generator.zero_grad()

        # Generate new fake text logits
        with torch.no_grad():
            outputs = generator(noise)
        fake_logits = outputs.logits.argmax(dim=-1).cuda()

        # Get discriminator's assessment of the newly generated fake data
        fake_output = discriminator(fake_logits.view(-1, 1).cuda())

        real_labels = torch.ones((fake_output.size(0), 1), dtype=torch.float).cuda()

        # Calculate the generator's loss

        lossG = criterion(fake_output.view(-1, 1), real_labels)
        lossG.backward()
        
        torch.nn.utils.clip_grad_norm_(generator.parameters(), max_norm=1.0)

        optimizerG.step()
        
        print(f'Epoch [{epoch}/{epochs}] Loss_D: {lossD.item():.4f} Loss_G: {lossG.item():.4f}')
        print("BATCH NUMBER " + str(batches))
        batches+=1
        del padded_inputs, real_text, noise, outputs, fake_logits, fake_logits_raw, real_labels, real_output, fake_labels, fake_output
        # gc.collect(generation=2)
        torch.cuda.empty_cache()

In [135]:
#Epoch [4/5] Loss_D: 1.2244 Loss_G: 0.5528

# 

In [46]:
inputs = tokenizer(positivePrompt, return_tensors='pt').to(device)
attention_mask = inputs['attention_mask'].to(device)

outputs = model.generate(
    inputs['input_ids'],
    do_sample=True,
    attention_mask=attention_mask,
    num_return_sequences=1,
    max_length=100,
    temperature = 0.9, 
    top_p = 0.9, 
    repetition_penalty=1.2,
    output_scores=True,
    return_dict_in_generate=True
)

# Decode the generated text
text = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
with torch.no_grad():
        # Forward pass to get logits
        initial_logits = model(outputs.sequences).logits
print("Generated Text:", text)

print("\n\n\n")

# Modify the generated text
modified_text = findTweets(text, True)
print("Modified Text:", modified_text)

# Tokenize the modified text
modified_inputs = tokenizer(modified_text, return_tensors='pt').to(device)
attention_mask = modified_inputs['attention_mask'].to(device)
length = len(tokenizer.encode(modified_text, add_special_tokens=True)) + 1

outputs = model.generate(
    modified_inputs['input_ids'],
    do_sample=True,
    attention_mask=attention_mask,
    num_return_sequences=1,
    max_length=length,
    temperature = 0.9, 
    top_p = 0.9,  
    repetition_penalty=1.2,
    output_scores=True,
    return_dict_in_generate=True
)

print("\n\n\n")

modified_text = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)

# new_logits = outputs.scores

with torch.no_grad():
        # Forward pass to get logits
        new_logits = model(outputs.sequences).logits

print("Generated Text:", modified_text)

print("Logits Shape:", new_logits)
print("Logits Shape:", initial_logits)

Generated Text: 
Generate a positive social media tweet on a specific topic. Ensure your tweet is enclosed in straight double quotation marks. Provide ONLY one tweet. The positive tweet should express enthusiasm or praise. 

Positive: "Thrilled to see the new community garden sprouting up! #GreenThumbsUp🌱 Community spirit at its finest!" (1/2) Your task now entails compos




Modified Text: Thrilled to see the new community garden sprouting up! #GreenThumbsUp🌱 Community spirit at its finest!




Generated Text: Thrilled to see the new community garden sprouting up! #GreenThumbsUp🌱 Community spirit at its finest!

Logits Shape: tensor([[[ 4.3703,  6.7406,  7.3559,  ...,  3.5807,  3.5774,  3.5800],
         [17.8003, 18.3170, 18.6253,  ..., 13.0327, 13.0342, 13.0326],
         [31.4694, 33.6264, 33.7634,  ..., 29.0450, 29.0487, 29.0510],
         ...,
         [26.2451, 27.1325, 28.0734,  ..., 22.7051, 22.7045, 22.7061],
         [27.5618, 27.3902, 30.5799,  ..., 24.3023, 24.3039, 24.302